In [1]:
!pip install transformers torch tqdm

In [2]:
import sys
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.optim import Adam
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm

/home/trainora/Desktop/MoryaAI/morya/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Add files to system for direct calling of class
sys.path.append('/kaggle/input/morya-83m')
print(sys.path)

from train import ChatData

['/home/trainora/Desktop/MoryaAI/server/app/api/v1/llms', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/trainora/Desktop/MoryaAI/morya/lib/python3.11/site-packages', '/kaggle/input/morya-83m']


In [4]:
# model_path = './distilgpt2'
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
special_tokens_dict = {'bos_token': '<startofstring>',
                       'eos_token': '<endofstring>',
                       'additional_special_tokens': ['<bot> :']}
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(50260, 768)

In [5]:
# Load dataset
chatData = ChatData('GPT-2-conversation_dataset.json', tokenizer)
sample_data = chatData[0] # debug line
print(sample_data)

(tensor([50257,  1374,   447,   247,    82,   340,  1016,    30,  1279, 13645,
        31175,  1312,  6004,   284,  4095,  2647,    13,   345,    30,  1279,
          437,   455,  1806,    29, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,

In [6]:
# Prepare DataLoader
data_loader = DataLoader(chatData, batch_size=16, shuffle=True)

In [7]:
# Correct device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)  # Correct method to move model to device

num_epochs = 5
optimizer = Adam(model.parameters(), lr=5e-5)

In [8]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(enumerate(data_loader), total=len(data_loader), desc=f'Epoch {epoch+1}/{num_epochs}')
    for batch_idx, batch in progress_bar:
        input_ids, attention_mask = [b.to(device) for b in batch]

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids.detach())
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        if (batch_idx + 1) % 100 == 0:  # Adjust the modulus value to control print frequency
            print(f'Epoch: {epoch+1}, Batch: {batch_idx+1}/{len(data_loader)}, Loss: {loss.item()}')

    avg_loss = total_loss / len(data_loader)
    print(f'Epoch: {epoch+1}, Average Loss: {avg_loss}')

Epoch 1/5:   0%|          | 0/1229 [00:00<?, ?it/s]

In [ ]:
model.save_pretrained('/kaggle/output/morya-83m')
tokenizer.save_pretrained('/kaggle/output/morya-83m')